In [74]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen
import re
import time
import requests

In [203]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [211]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [178]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

In [212]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [76]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [97]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

In [98]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

In [99]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

In [100]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

In [101]:
def get_profile(whole_string) : 
    if '"profile"' in whole_string:
        profile_string = re.findall(r'\"profile\"\:\{(.+?)\}', whole_string)[0]
    else:
        profile_string = re.findall(r'profile\:\{(.+?)\}', whole_string)[0]
    profile_info_dict = scraper_prototype(profile_string, 'profile')
    df4 = pd.DataFrame.from_dict(profile_info_dict, orient='index').transpose()
    return df4

In [234]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

In [235]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

In [246]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])

In [247]:
df = pd.concat([row,row0,row1,row2,row3,row4,row5,row6,row7,row8,row9,row10],ignore_index=True)

In [248]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,creator_small,creator_id,creator_name,creator_slug,creator_is_registered,creator_is_email_verified,creator_chosen_currency,creator_is_superbacker,creator_avatar,creator_small
0,Dragon Island,Dragon Island is a very horny comedy Visual No...,3000.0,16687.69,live,dragon-island,false,DK,Denmark,EUR,...,https://ksr-ugc.imgix.net/assets/036/000/112/3...,1144213587,Studio Selfcropping,dragonisland,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/00...,https://ksr-ugc.imgix.net/assets/036/000/112/3...
1,Supernova,"He has social anxiety, she has no filter. When...",1500.0,1500.5,live,supernova-2,false,GB,the United Kingdom,GBP,...,https://ksr-ugc.imgix.net/assets/036/224/571/e...,1641228307,Roshni Bohra,roshrosh942,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/22...,https://ksr-ugc.imgix.net/assets/036/224/571/e...
2,Make 100 - A Rocky Horror Enamel Pin ; Magnet ...,A limited edition enamel pin and magnet series...,400.0,1507.0,live,make-100-a-rocky-horror-enamel-pin-and-magnet-...,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/021/004/649/1...,1612525011,Fred Moreau,repo,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/021/00...,https://ksr-ugc.imgix.net/assets/021/004/649/1...
3,Shieldbreaker,A legendary viking warrior makes a deal with F...,2000.0,3043.32,live,shieldbreaker,false,AU,Australia,AUD,...,https://ksr-ugc.imgix.net/assets/035/851/338/d...,899554940,Mickey Scott,madbastard,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/035/85...,https://ksr-ugc.imgix.net/assets/035/851/338/d...
4,Edgar Allen Poe Enamel Pins,Enamel Pins featuring Edgar Allen Poe and His ...,600.0,1228.0,live,edgar-allen-poe-enamel-pins,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/007/429/143/f...,2120400576,Kevin Meinert,kem,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/007/42...,https://ksr-ugc.imgix.net/assets/007/429/143/f...
5,A Tiger's Tale : vol. 1,The Tao Te Ching meets The Jungle Book in this...,2222.0,6321.0,live,a-tigers-tale-the-graphic-novel-vol-1,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/029/756/000/d...,503461725,Patrick Lugo [@PLUGO or @PLUGOarts],plugoarts,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/029/75...,https://ksr-ugc.imgix.net/assets/029/756/000/d...
6,Spellcraft: A Primer on the Creation of Sorceries,Spellcraft is a solo journaling TRPG where you...,500.0,2322.0,live,spellcrafting-a-primer-on-the-creation-of-sorc...,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/036/161/602/1...,524924361,Kage Guenther,inkwyrmling,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/16...,https://ksr-ugc.imgix.net/assets/036/161/602/1...
7,Make 100 Unique Silver Stud Earrings,"Individual, hand patterned .925 sterling silve...",200.0,1299.0,live,make-100-unique-silver-stud-earrings,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/036/226/066/5...,1854969911,Canary Zindell,canaryzindell,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/22...,https://ksr-ugc.imgix.net/assets/036/226/066/5...
8,The Waltz (Feature Film Development),A single young trans woman teams up with a rel...,3500.0,3605.5,live,the-waltz-feature-film-development,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/007/339/866/5...,857310971,Ava Davis,theavadavis,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/007/33...,https://ksr-ugc.imgix.net/assets/007/339/866/5...
9,My Friend Toothy,A children's book about how to practice good d...,3500.0,10847.79,live,my-friend-toothy,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/035/951/930/1...,384894682,My Friend Toothy,myfriendtoothy,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/035/95...,h

In [187]:
for i in range(0,len(containers)) : 
    string =  to_string(containers[i])
    name = get_name(string)
    creator = get_creator(string)
    location = get_location(string)
    profile = get_profile(string)
    my_dict[i] = name,creator,location,profile
    df = pd.DataFrame(my_dict)
    final_df = pd.concat([final_df, df],ignore_index=True)
    i =+ 1

In [240]:
for i in range(1, len(containers)) : 
    final_df = row_flow(containers[0])
    #next_row = row_flow(containers[i])
    final_df = pd.concat([final_df, row_flow(containers[i])],ignore_index=True)
    i =+1

In [256]:
df['creator_small'][0]

KeyError: 0

In [243]:
len(containers)

12

In [259]:
creator_id = df[df['creator_id'][0]]

KeyError: 0